In [1]:
"""
To reduce the testing time of the program, the following code computes
the Difference of Gaussians(DoGs) for only one octave. However, 
by changing the value of sigma in the 'gauss' function, it can be used
to calculate DoGs for any octave.
"""

import numpy as np
import cv2

# Gaussian kernel
def gauss(l, sig):
    """
    creates gaussian kernel with side length l and a sigma of sig
    """

    ax = np.arange(-l // 2 + 1., l // 2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
 
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sig**2))

    return kernel / np.sum(kernel)

#T=gauss(7,5.)
T1=gauss(7, 1/(2**(1/2)))
T2=gauss(7, 1 )
T3=gauss(7, (2**(1/2)) )
T4=gauss(7, 2)
T5=gauss(7, 2*(2**(1/2)) )



In [2]:
img = cv2.imread('task2.jpg',0)

#img=(img[0::2,0::2])                       #  Image is resized by dropping alternative
                                           #  columns and rows to generate octave 2
#print(img.shape)                           #  Dimension (229,375) 

#img=(img[0::2,0::2])                       #  Half of the original Image is resized by
                                           #   dropping alternative columns and rows 
                                           #   to generate octave 3

#print(img.shape)                           # Dimension (115,188)

#img=(img[0::2,0::2])

#print(img.shape)                           # Dimension (58,94)

img1=img.copy();
img2=img.copy();
img3=img.copy();
img4=img.copy();
img5=img.copy();

images= [ img1,img2,img3,img4,img5 ]
scales = [ T1,T2,T3,T4,T5 ]
blured=[]
dog=[]

rows=img.shape[0];
columns=img.shape[1];
print(rows,columns)

zero_matrix=np.zeros((rows,columns),dtype=np.float)

#  convolve image and gaussian kernel
for i in range(len(images)):
    output=zero_matrix.copy()
    for row in range(0,rows-6):
        for column in range(0,columns-6):

            window=images[i][row:row+7,column:column+7]
            # print(window);
            sum1 = ( window * scales[i] ).sum();

            output[row+3][column+3]=sum1
    blured.append(output.copy())


458 750


In [3]:
for i in range(len(blured)-1):
    cv2.imwrite('octave4_img' + str(i) + '.jpg', blured[i])
    dog.append(blured[i+1]-blured[i])

cv2.imwrite('octave4_img' + str(4) + '.jpg',blured[4])

print(len(dog))    
#cv2.imshow('org',img)

# DoGs of 
for i in range(len(dog)):
    cv2.imwrite('dog' + str(i+1) + '_' + str(i) + str(4) +'.jpg' ,dog[i])



4


In [4]:
dog_x=dog[0].shape[0];
dog_y=dog[0].shape[1];

dog_matrix=np.zeros((dog_x,dog_y),dtype=np.float)
extrema = []
for i in range(1,len(dog)-1):
    for row in range(0,dog[0].shape[0]-2):
        for column in range(0,dog[0].shape[1]-2):
            
            middle = dog[i][row:row+3, column:column+3]
            first = dog[i-1][row:row+3, column:column+3]
            last = dog[i+1][row:row+3, column:column+3]
            
            pivot=middle[1][1]
            mini=pivot
            maxi=pivot
            
            for roww in range(middle.shape[0]):
                for coll in range(middle.shape[1]):
                    if(middle[roww][coll]<mini):
                        mini=middle[roww][coll]
                    if(first[roww][coll]<mini):
                        mini=first[roww][coll]
                    if(last[roww][coll]<mini):
                        mini=last[roww][coll]    
                    
                    if(middle[roww][coll]>maxi):
                        maxi=middle[roww][coll]
                    if(first[roww][coll]>maxi):
                        maxi=first[roww][coll]
                    if(last[roww][coll]>maxi):
                        maxi=last[roww][coll]    
                    
                    if(pivot<maxi and pivot>mini):
                        break;
                if(pivot<maxi and pivot>mini):
                    break;
            
            if( pivot==maxi or pivot==mini ):
                dog_matrix[row+1][column+1] = 255
            else:
                dog_matrix[row+1][column+1]=0
    extrema.append(dog_matrix.copy())

cv2.imwrite("extrema1_1.jpg",extrema[0])
cv2.imwrite("extrema2_1.jpg",extrema[1])

for i in range(extrema[0].shape[0]):
    for j in range(extrema[0].shape[1]):
        if(extrema[1][i][j]==255):
            img[i][j]=255
cv2.imwrite("whiteDotted.jpg",img)

counter=0


"""
We will start detecting points from third row and third column because the first 
row and column are unattended because of 3*3 gaussian mask. So,there will be more
white dots in the first and second row,column because of blur effect, which are 
not the keypoints

"""
for j in range(2,img.shape[1]):
    for i in range(2,img.shape[0]):
        if(counter==5):
            break
        
        if(img[i][j]==255):
            counter+=1
            print("Left most keypoint number " + str(counter)  )
            print("Detected at position: (" + str(i) + "," + str(j) + ")" )
    if(counter==5):
        break
    

cv2.waitKey(100000)

cv2.destroyAllWindows()

Left most keypoint number 1
Detected at position: (5,2)
Left most keypoint number 2
Detected at position: (6,2)
Left most keypoint number 3
Detected at position: (14,2)
Left most keypoint number 4
Detected at position: (22,2)
Left most keypoint number 5
Detected at position: (25,2)
